In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import os

In [2]:
def download_data(path):
    data = []
    for path_image in sorted(os.listdir(path=path)):
        image = Image.open(path + path_image)  # Открываем изображение.
        data.append(np.array(image))  # Загружаем пиксели.
    return data

In [4]:
X_train = download_data("./train/images/")
Y_train = download_data("./train/masks/")
X_test = download_data("./test/images/")
Y_test = download_data("./test/masks/")

In [28]:
X_train[0]

array([[[177, 181, 184],
        [205, 209, 212],
        [219, 222, 227],
        ...,
        [226, 229, 244],
        [230, 233, 250],
        [243, 246, 255]],

       [[217, 221, 224],
        [244, 248, 251],
        [252, 255, 255],
        ...,
        [229, 232, 247],
        [217, 220, 235],
        [208, 211, 226]],

       [[234, 238, 241],
        [252, 255, 255],
        [250, 253, 255],
        ...,
        [240, 242, 255],
        [237, 239, 252],
        [226, 228, 241]],

       ...,

       [[201, 198, 209],
        [221, 218, 229],
        [221, 215, 225],
        ...,
        [241, 234, 224],
        [255, 250, 240],
        [255, 251, 241]],

       [[194, 191, 202],
        [215, 212, 223],
        [223, 217, 227],
        ...,
        [222, 215, 205],
        [230, 223, 213],
        [223, 216, 206]],

       [[212, 209, 220],
        [195, 192, 203],
        [179, 173, 183],
        ...,
        [240, 233, 223],
        [255, 248, 238],
        [253, 246, 236]]

In [9]:
Y_train[0]

array([[[132,  41, 246, 255],
        [132,  41, 246, 255],
        [132,  41, 246, 255],
        ...,
        [132,  41, 246, 255],
        [132,  41, 246, 255],
        [132,  41, 246, 255]],

       [[132,  41, 246, 255],
        [132,  41, 246, 255],
        [132,  41, 246, 255],
        ...,
        [132,  41, 246, 255],
        [132,  41, 246, 255],
        [132,  41, 246, 255]],

       [[132,  41, 246, 255],
        [132,  41, 246, 255],
        [132,  41, 246, 255],
        ...,
        [132,  41, 246, 255],
        [132,  41, 246, 255],
        [132,  41, 246, 255]],

       ...,

       [[132,  41, 246, 255],
        [132,  41, 246, 255],
        [132,  41, 246, 255],
        ...,
        [132,  41, 246, 255],
        [132,  41, 246, 255],
        [132,  41, 246, 255]],

       [[132,  41, 246, 255],
        [132,  41, 246, 255],
        [132,  41, 246, 255],
        ...,
        [132,  41, 246, 255],
        [132,  41, 246, 255],
        [132,  41, 246, 255]],

       [[132

In [15]:
len(Y_train)

7

In [16]:
len(Y_train[0][0])

796

In [17]:
len(Y_train[0][0][0])

4

In [19]:
palette = {
    0: (60, 16, 152),  # Building
    1: (132, 41, 246),  # Land
    2: (110, 193, 228),  # Road
    3: (254, 221, 58),  # Vegetation
    4: (226, 169, 41),  # Water
    5: (155, 155, 155),  # Unlabeled
}

In [22]:
invert_palette = {v: k for k, v in palette.items()}

In [20]:
# сегментация нейронной сети в RGB изображение
def convert_to_color(arr_2d, palette=palette):
    """Numeric labels to RGB-color encoding"""
    arr_3d = np.zeros((arr_2d.shape[0], arr_2d.shape[1], 3), dtype=np.uint8)

    for c, i in palette.items():
        m = arr_2d == c
        arr_3d[m] = i

    return arr_3d

In [23]:
def convert_from_color(arr_3d, palette=invert_palette):
    """RGB-color encoding to grayscale labels"""
    arr_2d = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.uint8)

    arr_2d = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.int8)  # принадлежность каждого пикселя классу
    min_distance = (
        np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.float32) + 1000
    )  # расстояние до ближайшего класса для пикселей
    for c, i in palette.items():
        distance = np.sum((arr_3d - np.array(c).reshape(1, 1, 3)) ** 2, axis=-1) ** (
            1 / 2
        )  # ищем расстояние для каждого пикселя до проверяемого класса по евклиду рас-ие
        condition = min_distance > distance  # поиск элементов меньше min_distance
        min_distance[condition] = distance[condition]  # замена дистанции найденных элементов
        arr_2d[condition] = i  # замена класса найденных элементов

    for c, i in palette.items():
        m = np.all(arr_3d == np.array(c).reshape(1, 1, 3), axis=2)
        arr_2d[m] = i

    arr_2d = arr_2d.tolist()
    for i in range(len(arr_2d)):
        for j in range(len(arr_2d[0])):
            label = [0, 0, 0, 0, 0, 0]
            label[arr_2d[i][j]] = 1
            arr_2d[i][j] = label
    arr_2d = np.array(arr_2d)

    return arr_2d

In [29]:
X_train_pred = np.array(X_train).reshape([7, 644, 796, 3]) / 255
X_test_pred = np.array(X_test).reshape([2, 644, 796, 3]) / 255
Y_train_pred = []
for i in range(len(Y_train)):
    Y_train_pred.append(convert_from_color(Y_train[i][:, :, :3]))
Y_train_pred = np.array(Y_train_pred)
Y_test_pred = []
for i in range(len(Y_test)):
    Y_test_pred.append(convert_from_color(Y_test[i][:, :, :3]))
Y_test_pred = np.array(Y_test_pred)

In [30]:
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *

In [31]:
def unet_model(image_size, output_classes):

    # Входной слой
    input_layer = Input(shape=image_size + (3,))
    conv_1 = Conv2D(
        64, 4, activation=LeakyReLU(), strides=2, padding="same", kernel_initializer="glorot_normal", use_bias=False
    )(input_layer)
    # Сворачиваем
    conv_1_1 = Conv2D(
        128, 4, activation=LeakyReLU(), strides=2, padding="same", kernel_initializer="glorot_normal", use_bias=False
    )(conv_1)
    batch_norm_1 = BatchNormalization()(conv_1_1)

    # 2
    conv_2 = Conv2D(
        256, 4, activation=LeakyReLU(), strides=2, padding="same", kernel_initializer="glorot_normal", use_bias=False
    )(batch_norm_1)
    batch_norm_2 = BatchNormalization()(conv_2)

    # 3
    conv_3 = Conv2D(
        512, 4, activation=LeakyReLU(), strides=2, padding="same", kernel_initializer="glorot_normal", use_bias=False
    )(batch_norm_2)
    batch_norm_3 = BatchNormalization()(conv_3)

    # 4
    conv_4 = Conv2D(
        512, 4, activation=LeakyReLU(), strides=2, padding="same", kernel_initializer="glorot_normal", use_bias=False
    )(batch_norm_3)
    batch_norm_4 = BatchNormalization()(conv_4)

    # 5
    conv_5 = Conv2D(
        512, 4, activation=LeakyReLU(), strides=2, padding="same", kernel_initializer="glorot_normal", use_bias=False
    )(batch_norm_4)
    batch_norm_5 = BatchNormalization()(conv_5)

    # 6
    conv_6 = Conv2D(
        512, 4, activation=LeakyReLU(), strides=2, padding="same", kernel_initializer="glorot_normal", use_bias=False
    )(batch_norm_5)
    # Разворачиваем
    # 1
    up_1 = Concatenate()(
        [
            Conv2DTranspose(
                512, 4, activation="relu", strides=2, padding="same", kernel_initializer="glorot_normal", use_bias=False
            )(conv_6),
            conv_5,
        ]
    )
    batch_up_1 = BatchNormalization()(up_1)

    # Добавим Dropout от переобучения
    batch_up_1 = Dropout(0.25)(batch_up_1)

    # 2
    up_2 = Concatenate()(
        [
            Conv2DTranspose(
                512, 4, activation="relu", strides=2, padding="same", kernel_initializer="glorot_normal", use_bias=False
            )(batch_up_1),
            conv_4,
        ]
    )
    batch_up_2 = BatchNormalization()(up_2)
    batch_up_2 = Dropout(0.25)(batch_up_2)

    # 3
    up_3 = Concatenate()(
        [
            Conv2DTranspose(
                512, 4, activation="relu", strides=2, padding="same", kernel_initializer="glorot_normal", use_bias=False
            )(batch_up_2),
            conv_3,
        ]
    )
    batch_up_3 = BatchNormalization()(up_3)
    batch_up_3 = Dropout(0.25)(batch_up_3)

    # 4
    up_4 = Concatenate()(
        [
            Conv2DTranspose(
                256, 4, activation="relu", strides=2, padding="same", kernel_initializer="glorot_normal", use_bias=False
            )(batch_up_3),
            conv_2,
        ]
    )
    batch_up_4 = BatchNormalization()(up_4)

    # 5
    up_5 = Concatenate()(
        [
            Conv2DTranspose(
                128, 4, activation="relu", strides=2, padding="same", kernel_initializer="glorot_normal", use_bias=False
            )(batch_up_4),
            conv_1_1,
        ]
    )
    batch_up_5 = BatchNormalization()(up_5)

    # 6
    up_6 = Concatenate()(
        [
            Conv2DTranspose(
                64, 4, activation="relu", strides=2, padding="same", kernel_initializer="glorot_normal", use_bias=False
            )(batch_up_5),
            conv_1,
        ]
    )
    batch_up_6 = BatchNormalization()(up_6)

    # Выходной слой
    output_layer = Conv2DTranspose(
        output_classes, 4, activation="sigmoid", strides=2, padding="same", kernel_initializer="glorot_normal"
    )(batch_up_6)

    return Model(inputs=input_layer, outputs=output_layer)